# Data Analysis (Preliminary)
#### John R. Starr; jrs294@pitt.edu
Time for some analysis! Let's load in the usual modules and then begin our analysis.

In [1]:
import nltk
import numpy as np
import pandas as pd

In [2]:
df = pd.read_pickle('ordered_only_df.pkl')

A breakdown on what we've got:

In [3]:
df.head()

,ID,Eng,Far,Eng_Tok,Far_Tok,Eng_Len,Far_Len,Eng_Types,Far_Types,Far_POS,Far_Chunks,Eng_POS,Eng_Chunks,Word_Order
4,5,stop please stop,دست نگه داريد خواهش ميکنم دست نگه داريد,"[stop, please, stop]","[دست, نگه, داريد, خواهش, ميکنم, دست, نگه, داريد]",3,8,"{please, stop}","{ميکنم, دست, داريد, نگه, خواهش}","[(دست, N), (نگه, N), (داريد, V), (خواهش, Ne), ...",[دست NP] [نگه داريد VP] [خواهش ميکنم دست NP] [...,"[(stop, JJ), (please, NN), (stop, VB)]","[[(stop, JJ), (please, NN)], [[('stop', 'VB')]]]",SVO
8,9,god damn it put that down,لعنت به تو اونو بذار زمين,"[god, damn, it, put, that, down]","[لعنت, به, تو, اونو, بذار, زمين]",6,6,"{it, down, that, put, damn, god}","{به, زمين, اونو, بذار, تو, لعنت}","[(لعنت, N), (به, P), (تو, PRO), (اونو, PRO), (...",[لعنت NP] [به PP] [تو NP] [اونو NP] [بذار VP] ...,"[(god, NN), (damn, VBZ), (it, PRP), (put, VBD)...","[[(god, NN)], [[('damn', 'VBZ')]], (it, PRP), ...",SOV
10,11,its the last feed weve got,اين آخرين علوفه اي بود که ما داشتيم,"[its, the, last, feed, weve, got]","[اين, آخرين, علوفه, اي, بود, که, ما, داشتيم]",6,8,"{the, last, its, got, feed, weve}","{ما, داشتيم, که, بود, علوفه, آخرين, اي, اين}","[(اين, Ne), (آخرين, NUM), (علوفه, N), (اي, N),...",[اين آخرين علوفه NP] [اي NP] [بود VP] که [ما N...,"[(its, PRP$), (the, DT), (last, JJ), (feed, NN...","[(its, PRP$), [(the, DT), (last, JJ), (feed, N...",SOV
14,15,you lied to me dan,تو به من دروغ گفتي دن,"[you, lied, to, me, dan]","[تو, به, من, دروغ, گفتي, دن]",5,6,"{dan, me, to, lied, you}","{دن, دروغ, گفتي, من, به, تو}","[(تو, PRO), (به, P), (من, PRO), (دروغ, AJ), (گ...",[تو NP] [به PP] [من NP] [دروغ NP] [گفتي VP] دن,"[(you, PRP), (lied, VBD), (to, TO), (me, PRP),...","[(you, PRP), [[('lied', 'VBD')]], (to, TO), (m...",SOV
15,16,you told me we made payments to hollander we did,تو به من گفتي قرضمونو به هلندر پرداخت کرديم ما...,"[you, told, me, we, made, payments, to, hollan...","[تو, به, من, گفتي, قرضمونو, به, هلندر, پرداخت,...",10,12,"{did, me, to, told, made, we, payments, hollan...","{قرضمونو, ما, گفتي, به, من, هلندر, کرديم, پردا...","[(تو, PRO), (به, P), (من, PRO), (گفتي, V), (قر...",[تو NP] [به PP] [من NP] [گفتي VP] [قرضمونو NP]...,"[(you, PRP), (told, VBD), (me, PRP), (we, PRP)...","[(you, PRP), [[('told', 'VBD')]], (me, PRP), (...",SOV


In [4]:
df.reset_index(drop = True, inplace = True)

In [5]:
print(len(df))
print()
print(df['Word_Order'].value_counts())

76715

SOV    52756
SVO    23959
Name: Word_Order, dtype: int64


Well, we have over double the number of SOV sentences than we do SVO sentences -- this isn't a bad thing, since Persian is underlyingly SOV. Let's separate the different orderings into their own respective DFs and then examine some of the structures that we find in both.

In [6]:
#  Sorting by the values
df.sort_values(by=['Word_Order'], inplace = True)

In [7]:
# Creating new DFs
sov_df = df.loc[df.Word_Order == 'SOV']
svo_df = df.loc[df.Word_Order == 'SVO']

In [8]:
# Making sure they are the proper lengths
print(len(sov_df))
print(len(svo_df))

52756
23959


Cool! Let's start with the SOV data first.

### SOV Data
Just a snippet of what we're working with:

In [9]:
sov_df.head()

,ID,Eng,Far,Eng_Tok,Far_Tok,Eng_Len,Far_Len,Eng_Types,Far_Types,Far_POS,Far_Chunks,Eng_POS,Eng_Chunks,Word_Order
38357,276474,i will tell your wife he is your son,من به زنت ميگم ، اون پسر توئه,"[i, will, tell, your, wife, he, is, your, son]","[من, به, زنت, ميگم, ،, اون, پسر, توئه]",9,8,"{will, tell, he, is, wife, your, i, son}","{توئه, من, به, ميگم, ،, اون, زنت, پسر}","[(من, PRO), (به, P), (زنت, N), (ميگم, V), (،, ...",[من NP] [به PP] [زنت NP] [ميگم VP] ، [اون پسر ...,"[(i, NN), (will, MD), (tell, VB), (your, PRP$)...","[[(i, NN)], (will, MD), [[('tell', 'VB')]], (y...",SOV
37914,273398,someones trying to shoot us,يكي ميخواد به ما شليك كنه,"[someones, trying, to, shoot, us]","[يكي, ميخواد, به, ما, شليك, كنه]",5,6,"{us, to, shoot, trying, someones}","{ما, كنه, به, يكي, ميخواد, شليك}","[(يكي, DET), (ميخواد, N), (به, P), (ما, PRO), ...",[يكي ميخواد NP] [به PP] [ما NP] [شليك VP] كنه,"[(someones, NNS), (trying, VBG), (to, TO), (sh...","[(someones, NNS), [[('trying', 'VBG')]], (to, ...",SOV
64714,479066,sa yong's right. is there a ransom demand?,حق با سا يونگ است. ممكنه كه نقشه ديگه اي هم در...,"[sa, yong, 's, right, ., is, there, a, ransom,...","[حق, با, سا, يونگ, است, ., ممكنه, كه, نقشه, دي...",11,16,"{demand, yong, there, ransom, a, is, 's, right...","{كه, سا, ممكنه, در, حق, يونگ, ديگه, اي, داشته,...","[(حق, N), (با, P), (سا, N), (يونگ, N), (است, V...",[حق NP] [با PP] [سا يونگ NP] [است VP] . [ممكنه...,"[(sa, NN), (yong, PRP), ('s, POS), (right, JJ)...","[[(sa, NN)], (yong, PRP), ('s, POS), [(right, ...",SOV
37917,273426,actually a thank you would be nice,درحقيقت يک تشكر ميتونه خوب باشه,"[actually, a, thank, you, would, be, nice]","[درحقيقت, يک, تشكر, ميتونه, خوب, باشه]",7,6,"{nice, be, thank, a, actually, you, would}","{باشه, خوب, ميتونه, تشكر, يک, درحقيقت}","[(درحقيقت, Ne), (يک, N), (تشكر, P), (ميتونه, N...",[درحقيقت يک NP] [تشكر PP] [ميتونه خوب NP] [باش...,"[(actually, RB), (a, DT), (thank, NN), (you, P...","[(actually, RB), [(a, DT), (thank, NN)], (you,...",SOV
37918,273430,i dont want an axe thats crazy why would you o...,من يک تبر نميخوام اين احمقانه است,"[i, dont, want, an, axe, thats, crazy, why, wo...","[من, يک, تبر, نميخوام, اين, احمقانه, است]",14,7,"{me, dont, why, crazy, offer, thats, axe, i, y...","{نميخوام, من, تبر, احمقانه, يک, اين, است}","[(من, PRO), (يک, NUM), (تبر, N), (نميخوام, V),...",[من NP] [يک تبر NP] [نميخوام VP] [اين احمقانه ...,"[(i, NNS), (dont, VBP), (want, VBP), (an, DT),...","[(i, NNS), [[('dont', 'VBP')]], [[('want', 'VB...",SOV


In [10]:
# Making the indexes easier to work with
sov_df.reset_index(drop = True, inplace = True)

Just out of curiousity, I want to see if there's any big difference between the average lengths of the English and Persian SOV sentences:

In [11]:
avg_eng_len = []
for ln in sov_df['Eng_Len']:
    avg_eng_len.append(ln)
print('The average English sentence length in the SOV file is...', (sum(avg_eng_len))/len(sov_df))

The average English sentence length in the SOV file is... 8.479926453863067


In [12]:
avg_far_len = []
for ln in sov_df['Far_Len']:
    avg_far_len.append(ln)
print('The average Persian sentence length in the SOV file is...', (sum(avg_far_len))/len(sov_df))

The average Persian sentence length in the SOV file is... 8.163753885813936


Not too big of a difference, it seems. I figured this wouldn't have much of an impact, but it was worth at least checking.

Let's see how sentences involving the word phrase "is like" fared! This phrase is common (and easily searchable) way of looking for similes.

In [13]:
# Question for graders: how can I do this more efficiently?
is_like_bool = []
for line in sov_df['Eng']:
    if 'is like' in line:
        is_like_bool.append(True)
    else:
        is_like_bool.append(False)
has_like = pd.Series(is_like_bool)

In [14]:
like_sov_df = sov_df[has_like]

In [15]:
print(len(like_sov_df))
like_sov_df.head(10)

35


,ID,Eng,Far,Eng_Tok,Far_Tok,Eng_Len,Far_Len,Eng_Types,Far_Types,Far_POS,Far_Chunks,Eng_POS,Eng_Chunks,Word_Order
110,272686,capricorns castle is like a fivestar hotel,قلعه كاپريكورن مثل يک هتل 5 ستاره است,"[capricorns, castle, is, like, a, fivestar, ho...","[قلعه, كاپريكورن, مثل, يک, هتل, 5, ستاره, است]",7,8,"{fivestar, hotel, capricorns, castle, a, is, l...","{كاپريكورن, 5, مثل, ستاره, قلعه, يک, هتل, است}","[(قلعه, Ne), (كاپريكورن, N), (مثل, ADVe), (يک,...",[قلعه كاپريكورن NP] [مثل PP] [يک هتل NP] [5 ست...,"[(capricorns, NNS), (castle, NN), (is, VBZ), (...","[(capricorns, NNS), [(castle, NN)], [[('is', '...",SOV
1434,478344,you little rascal who is like a mouse where do...,هي آدم حقه باز، فكر مي كني كجا داري ميري؟,"[you, little, rascal, who, is, like, a, mouse,...","[هي, آدم, حقه, باز،, فكر, مي, كني, كجا, داري, ...",16,10,"{little, 're, ?, a, is, who, rascal, like, mou...","{آدم, مي, حقه, فكر, هي, كجا, ميري؟, باز،, داري...","[(هي, CONJ), (آدم, Ne), (حقه, AJ), (باز،, Pe),...",[هي آدم حقه NP] [باز، PP] [فكر مي NP] [كني VP]...,"[(you, PRP), (little, JJ), (rascal, JJ), (who,...","[(you, PRP), [(little, JJ), (rascal, JJ)], (wh...",SOV
3810,302406,that woman is like an angelic goddess who only...,اون زن مثل يک فرشته الهيه که به اين دنيا مهربو...,"[that, woman, is, like, an, angelic, goddess, ...","[اون, زن, مثل, يک, فرشته, الهيه, که, به, اين, ...",14,12,"{woman, only, that, to, angelic, brings, is, w...","{به, الهيه, ميبخشه, که, زن, مثل, اون, دنيا, مه...","[(اون, DET), (زن, N), (مثل, ADVe), (يک, NUM), ...",[اون زن NP] [مثل PP] [يک فرشته NP] [الهيه VP] ...,"[(that, DT), (woman, NN), (is, VBZ), (like, IN...","[[(that, DT), (woman, NN)], [[('is', 'VBZ')], ...",SOV
4892,304178,for sunlight is like gold,For sunlight is like gold براي نور خورشيد که ب...,"[for, sunlight, is, like, gold]","[For, sunlight, is, like, gold, براي, نور, خور...",5,12,"{sunlight, gold, is, like, for}","{به, که, نور, gold, sunlight, is, خورشيد, like...","[(For, RES), (sunlight, RES), (is, RES), (like...",[For sunlight is like gold NP] [براي PP] [نور ...,"[(for, IN), (sunlight, NN), (is, VBZ), (like, ...","[[[('for', 'IN')], [('sunlight', 'NN')]], [[('...",SOV
5098,306144,a crime scene is like a code,صحنه جرم رازهاي زيادي توش داره,"[a, crime, scene, is, like, a, code]","[صحنه, جرم, رازهاي, زيادي, توش, داره]",7,6,"{crime, is, a, like, code, scene}","{زيادي, توش, جرم, داره, صحنه, رازهاي}","[(صحنه, Ne), (جرم, N), (رازهاي, V), (زيادي, AD...",[صحنه NP] [جرم NP] [رازهاي VP] [زيادي ADVP] [ت...,"[(a, DT), (crime, NN), (scene, NN), (is, VBZ),...","[[(a, DT), (crime, NN), (scene, NN)], [[('is',...",SOV
7065,494907,power is like grains of sand.,قدرت مثل دانه هاي شنه,"[power, is, like, grains, of, sand, .]","[قدرت, مثل, دانه, هاي, شنه]",7,5,"{of, like, is, sand, power, grains, .}","{قدرت, مثل, دانه, شنه, هاي}","[(قدرت, N), (مثل, ADVe), (دانه, N), (هاي, V), ...",[قدرت NP] [مثل PP] [دانه NP] [هاي VP] [شنه NP],"[(power, NN), (is, VBZ), (like, IN), (grains, ...","[[(power, NN)], [[('is', 'VBZ')]], [(like, IN)...",SOV
10086,206774,i know what it is like to try and scrape by wi...,من ميدونم كه زندگي بدون پشتيبان چه جون كندني ميشه,"[i, know, what, it, is, like, to, try, and, sc...","[من, ميدونم, كه, زندگي, بدون, پشتيبان, چه, جون...",14,10,"{it, and, to, danna, without, a, is, scrape, l...","{كه, ميدونم, من, بدون, ميشه, كندني, پشتيبان, ز...","[(من, PRO), (ميدونم, Ne), (كه, DET), (زندگي, N...",[من NP] [ميدونم كه زندگي NP] [بدون PP] [پشتيبا...,"[(i, NN), (know, VBP), (what, WP), (it, PRP), ...","[[(i, NN)], [[('know', 'VBP')]], (what, WP), (...",SOV
13367,488580,it is like leader who worked so hard with his ...,اين مثل سرپرست هست، کسي که سخت کار مي کنه تا ش...,"[it, is, like, leader, who, worked, so, hard, ...","[اين, مثل, سرپرست, هست،, کسي, که, سخت, کار, مي...",15,15,"{it, swords, with, to, worked, like, is, who, ...","{هست،, سرپرست, که, مثل, مي, درست, شمشير, تا, ک...","[(اين, N), (مثل, ADVe), (سرپرست, Ne), (هست،, N...",[اين NP] [مثل PP] [سرپرست هست، NP] [کسي VP] که...,"[(it, PRP), (is

Persian's functional equivalents to the words "like" and "that" are "مثل" and "که" respectively. If you look closely at the data, nearly all the Persian sentences in this dataset use those words! And, conveniently enough, you see that those same sentences maintain the SOV structure that we anticipated before.

NOTE: More analysis needs to be done!

Now for the SVO data.

### SVO Data
Similar to what we did for the SOV data, let's flash a little bit of what we're working with:

In [16]:
svo_df.head()

,ID,Eng,Far,Eng_Tok,Far_Tok,Eng_Len,Far_Len,Eng_Types,Far_Types,Far_POS,Far_Chunks,Eng_POS,Eng_Chunks,Word_Order
59618,434130,what the hell are you doing?,تو داري چه غلطي ميكني؟,"[what, the, hell, are, you, doing, ?]","[تو, داري, چه, غلطي, ميكني؟]",7,5,"{the, doing, are, you, what, hell, ?}","{غلطي, ميكني؟, تو, داري, چه}","[(تو, PRO), (داري, V), (چه, DET), (غلطي, N), (...",[تو NP] [داري VP] [چه غلطي ميكني؟ NP],"[(what, WP), (the, DT), (hell, NN), (are, VBP)...","[(what, WP), [(the, DT), (hell, NN)], [[('are'...",SVO
68509,501157,as you know this is a war to drive the han arm...,شما مي دونيد اين يه جنگ هست که مي خوايم ارتش ه...,"[as, you, know, this, is, a, war, to, drive, t...","[شما, مي, دونيد, اين, يه, جنگ, هست, که, مي, خو...",17,21,"{they, 've, army, the, war, to, as, a, is, han...","{مي, دونيد, بيرون, کشيدند, بعد, يه, جنگ, از, ک...","[(شما, PRO), (مي, N), (دونيد, V), (اين, N), (ي...",[شما NP] [مي دونيد VP] [اين NP] [يه PP] [جنگ N...,"[(as, IN), (you, PRP), (know, VBP), (this, DT)...","[[(as, IN)], (you, PRP), [[('know', 'VBP')], [...",SVO
66936,493066,i will do whatever your highness say but if so...,هر چه بخواهيد انجام مي دم اما اگر اتفاقي براي ...,"[i, will, do, whatever, your, highness, say, b...","[هر, چه, بخواهيد, انجام, مي, دم, اما, اگر, اتف...",14,12,"{will, but, whatever, to, highness, happens, b...","{انجام, اما, مي, بخواهيد, فرزندم, اتفاقي, بياف...","[(هر, DET), (چه, N), (بخواهيد, V), (انجام, Ne)...",[هر چه NP] [بخواهيد VP] [انجام مي NP] [دم VP] ...,"[(i, NN), (will, MD), (do, VB), (whatever, WDT...","[[(i, NN)], (will, MD), [[('do', 'VB')]], (wha...",SVO
66966,493312,general heuk chi has requested to see you.,ژنرال هيوك چي تقاضا داره شما را ببينه,"[general, heuk, chi, has, requested, to, see, ...","[ژنرال, هيوك, چي, تقاضا, داره, شما, را, ببينه]",9,8,"{to, has, general, requested, see, ., you, chi...","{ژنرال, هيوك, تقاضا, چي, شما, ببينه, داره, را}","[(ژنرال, N), (هيوك, CONJ), (چي, DET), (تقاضا, ...",[ژنرال هيوك چي تقاضا NP] [داره VP] [شما NP] [ر...,"[(general, JJ), (heuk, NN), (chi, NN), (has, V...","[[(general, JJ), (heuk, NN), (chi, NN)], [[('h...",SVO
58350,425438,i think they can be the alternatives for you.,فكر كنم اونها تنها چار? تو باشند,"[i, think, they, can, be, the, alternatives, f...","[فكر, كنم, اونها, تنها, چار, ?, تو, باشند]",10,8,"{they, the, you, think, alternatives, can, for...","{باشند, تنها, اونها, كنم, فكر, تو, چار, ?}","[(فكر, N), (كنم, V), (اونها, PRO), (تنها, ADV)...",[فكر NP] [كنم VP] [اونها NP] [تنها چار NP] ? [...,"[(i, NN), (think, VBP), (they, PRP), (can, MD)...","[[(i, NN)], [[('think', 'VBP')]], (they, PRP),...",SVO


In [17]:
# Making the indexes easier to work with
svo_df.reset_index(drop = True, inplace = True)

In [18]:
avg_eng_len2 = []
for ln in svo_df['Eng_Len']:
    avg_eng_len2.append(ln)
print('The average English sentence length in the SOV file is...', (sum(avg_eng_len2))/len(svo_df))

The average English sentence length in the SOV file is... 8.849826787428524


In [19]:
avg_far_len2 = []
for ln in svo_df['Far_Len']:
    avg_far_len2.append(ln)
print('The average Persian sentence length in the SOV file is...', (sum(avg_far_len2))/len(svo_df))

The average Persian sentence length in the SOV file is... 8.371760090154012


In [20]:
is_like_bool = []
for line in svo_df['Eng']:
    if 'is like' in line:
        is_like_bool.append(True)
    else:
        is_like_bool.append(False)
has_like = pd.Series(is_like_bool)

In [21]:
like_svo_df = svo_df[has_like]

In [22]:
print(len(like_svo_df))
like_svo_df

13


,ID,Eng,Far,Eng_Tok,Far_Tok,Eng_Len,Far_Len,Eng_Types,Far_Types,Far_POS,Far_Chunks,Eng_POS,Eng_Chunks,Word_Order
94,434104,rounds is like being on a game show.,اين ويزيتها شبيه اغاز يه مسابقه تلوزيونيه,"[rounds, is, like, being, on, a, game, show, .]","[اين, ويزيتها, شبيه, اغاز, يه, مسابقه, تلوزيونيه]",9,7,"{a, is, show, game, like, rounds, being, on, .}","{مسابقه, اغاز, يه, ويزيتها, شبيه, اين, تلوزيونيه}","[(اين, DET), (ويزيتها, N), (شبيه, V), (اغاز, N...",[اين ويزيتها NP] [شبيه VP] [اغاز يه مسابقه تلو...,"[(rounds, NNS), (is, VBZ), (like, IN), (being,...","[(rounds, NNS), [[('is', 'VBZ')]], [(like, IN)...",SVO
4192,434767,that song is like a virus.,اين آهنگه مثله خوره ميمونه,"[that, song, is, like, a, virus, .]","[اين, آهنگه, مثله, خوره, ميمونه]",7,5,"{virus, that, a, is, like, song, .}","{اين, مثله, ميمونه, آهنگه, خوره}","[(اين, N), (آهنگه, V), (مثله, Ne), (خوره, Ne),...",[اين NP] [آهنگه VP] [مثله خوره ميمونه NP],"[(that, DT), (song, NN), (is, VBZ), (like, IN)...","[[(that, DT), (song, NN)], [[('is', 'VBZ')], [...",SVO
7740,124304,this is like shoe art,اين هست هنر کفش,"[this, is, like, shoe, art]","[اين, هست, هنر, کفش]",5,4,"{like, is, art, shoe, this}","{هنر, اين, هست, کفش}","[(اين, N), (هست, V), (هنر, Ne), (کفش, N)]",[اين NP] [هست VP] [هنر کفش NP],"[(this, DT), (is, VBZ), (like, IN), (shoe, JJ)...","[[(this, DT)], [[('is', 'VBZ')], [(P like/IN),...",SVO
8347,190435,christmas is like me racing back and forth bet...,کريسمس توي کورس من بين مامان و بابا خلاصه ميشه,"[christmas, is, like, me, racing, back, and, f...","[کريسمس, توي, کورس, من, بين, مامان, و, بابا, خ...",12,10,"{christmas, forth, racing, and, me, back, like...","{خلاصه, من, توي, ميشه, کريسمس, مامان, بين, و, ...","[(کريسمس, N), (توي, V), (کورس, Ne), (من, PRO),...",[کريسمس NP] [توي VP] [کورس من NP] [بين VP] [ما...,"[(christmas, NN), (is, VBZ), (like, IN), (me, ...","[[(christmas, NN)], [[('is', 'VBZ')]], [(like,...",SVO
9775,183359,our love is like a red red rose,عشقمون شبيه يک گل رز قرمزه,"[our, love, is, like, a, red, red, rose]","[عشقمون, شبيه, يک, گل, رز, قرمزه]",8,6,"{our, love, red, a, is, rose, like}","{عشقمون, گل, شبيه, قرمزه, رز, يک}","[(عشقمون, N), (شبيه, V), (يک, NUM), (گل, Ne), ...",[عشقمون NP] [شبيه VP] [يک گل رز قرمزه NP],"[(our, PRP$), (love, NN), (is, VBZ), (like, IN...","[(our, PRP$), [(love, NN)], [[('is', 'VBZ')], ...",SVO
10643,34687,living below is like living in space you dont ...,زندگي اين پايين شبيه زندگي تو فضاست نبايد زياد...,"[living, below, is, like, living, in, space, y...","[زندگي, اين, پايين, شبيه, زندگي, تو, فضاست, نب...",13,11,"{mistakes, dont, many, in, very, get, below, l...","{زياد, اشتباه, پايين, شبيه, زندگي, اين, تو, فض...","[(زندگي, Ne), (اين, DET), (پايين, N), (شبيه, V...",[زندگي اين پايين NP] [شبيه VP] [زندگي تو NP] [...,"[(living, VBG), (below, NN), (is, VBZ), (like,...","[[[('living', 'VBG')], [('below', 'NN')]], [[(...",SVO
12867,82480,to have an indecent behavior is like trampling...,رفتار هاي شرم آور درست شبيه اينه كه خون شهدا ر...,"[to, have, an, indecent, behavior, is, like, t...","[رفتار, هاي, شرم, آور, درست, شبيه, اينه, كه, خ...",14,13,"{behavior, blood, martyrs, the, of, to, trampl...","{شرم, پايمال, كه, رفتار, شهدا, درست, شبيه, اين...","[(رفتار, N), (هاي, V), (شرم, N), (آور, V), (در...",[رفتار NP] [هاي VP] [شرم آور VP] [درست شبيه اي...,"[(to, TO), (have, VB), (an, DT), (indecent, JJ...","[(to, TO), [[('have', 'VB')], [('an', 'DT'), (...",SVO
14413,81908,dying as a martyr is like injecting blood in t...,مردن بعنوان شهيد شبيه تزريق خون در شَريانه جام...,"[dying, as, a, martyr, is, like, injecting, bl...","[مردن, بعنوان, شهيد, شبيه, تزريق, خون, در, شَر...",13,10,"{society, blood, the, of, in, injecting, marty...","{مردن, شَريانه, ست, شهيد, در, شبيه, تزريق, جام...","[(مردن, N), (بعنوان, Pe), (شهيد, N), (شبيه, V)...",[مردن NP] [بعنوان شهيد شبيه VP] [تزريق خون NP]...,"[(dying, VBG), (as, IN), (a, DT), (martyr, NN)...","[[[('dying', 'VBG')], [(P as/IN), (NP a/DT mar...",SVO
14498,72418,you kidding me this stuff is like wa

As mentioned before, مثل and که are the functional words here that indicate an _obvious_ simile. However, only three of the sentences in this DF use those functional words! These sentences still translate the idea of a simile, but convert it to a more metaphorical approach. This approach may have an influence on why the structure of the translation is SVO...! However, this is definitely not a conclusion I can make quite yet -- there's much, much, _much_ more to do! 

Plan for the future:
- Build some sort of function that will help me sift through the data faster and search for keywords
- Determine what factors may influence a SVO word order to appear over an SOV
    - Examine functional words first, and then look for trends
    - Machine learning?
- Potentially incorporate a machine learning model from this data to predict whether or not a sentence will be SOV or SVO?

## After PR3:
#### Looking at the SOV Texts Above:

In [23]:
like_sov_df.head()

,ID,Eng,Far,Eng_Tok,Far_Tok,Eng_Len,Far_Len,Eng_Types,Far_Types,Far_POS,Far_Chunks,Eng_POS,Eng_Chunks,Word_Order
110,272686,capricorns castle is like a fivestar hotel,قلعه كاپريكورن مثل يک هتل 5 ستاره است,"[capricorns, castle, is, like, a, fivestar, ho...","[قلعه, كاپريكورن, مثل, يک, هتل, 5, ستاره, است]",7,8,"{fivestar, hotel, capricorns, castle, a, is, l...","{كاپريكورن, 5, مثل, ستاره, قلعه, يک, هتل, است}","[(قلعه, Ne), (كاپريكورن, N), (مثل, ADVe), (يک,...",[قلعه كاپريكورن NP] [مثل PP] [يک هتل NP] [5 ست...,"[(capricorns, NNS), (castle, NN), (is, VBZ), (...","[(capricorns, NNS), [(castle, NN)], [[('is', '...",SOV
1434,478344,you little rascal who is like a mouse where do...,هي آدم حقه باز، فكر مي كني كجا داري ميري؟,"[you, little, rascal, who, is, like, a, mouse,...","[هي, آدم, حقه, باز،, فكر, مي, كني, كجا, داري, ...",16,10,"{little, 're, ?, a, is, who, rascal, like, mou...","{آدم, مي, حقه, فكر, هي, كجا, ميري؟, باز،, داري...","[(هي, CONJ), (آدم, Ne), (حقه, AJ), (باز،, Pe),...",[هي آدم حقه NP] [باز، PP] [فكر مي NP] [كني VP]...,"[(you, PRP), (little, JJ), (rascal, JJ), (who,...","[(you, PRP), [(little, JJ), (rascal, JJ)], (wh...",SOV
3810,302406,that woman is like an angelic goddess who only...,اون زن مثل يک فرشته الهيه که به اين دنيا مهربو...,"[that, woman, is, like, an, angelic, goddess, ...","[اون, زن, مثل, يک, فرشته, الهيه, که, به, اين, ...",14,12,"{woman, only, that, to, angelic, brings, is, w...","{به, الهيه, ميبخشه, که, زن, مثل, اون, دنيا, مه...","[(اون, DET), (زن, N), (مثل, ADVe), (يک, NUM), ...",[اون زن NP] [مثل PP] [يک فرشته NP] [الهيه VP] ...,"[(that, DT), (woman, NN), (is, VBZ), (like, IN...","[[(that, DT), (woman, NN)], [[('is', 'VBZ')], ...",SOV
4892,304178,for sunlight is like gold,For sunlight is like gold براي نور خورشيد که ب...,"[for, sunlight, is, like, gold]","[For, sunlight, is, like, gold, براي, نور, خور...",5,12,"{sunlight, gold, is, like, for}","{به, که, نور, gold, sunlight, is, خورشيد, like...","[(For, RES), (sunlight, RES), (is, RES), (like...",[For sunlight is like gold NP] [براي PP] [نور ...,"[(for, IN), (sunlight, NN), (is, VBZ), (like, ...","[[[('for', 'IN')], [('sunlight', 'NN')]], [[('...",SOV
5098,306144,a crime scene is like a code,صحنه جرم رازهاي زيادي توش داره,"[a, crime, scene, is, like, a, code]","[صحنه, جرم, رازهاي, زيادي, توش, داره]",7,6,"{crime, is, a, like, code, scene}","{زيادي, توش, جرم, داره, صحنه, رازهاي}","[(صحنه, Ne), (جرم, N), (رازهاي, V), (زيادي, AD...",[صحنه NP] [جرم NP] [رازهاي VP] [زيادي ADVP] [ت...,"[(a, DT), (crime, NN), (scene, NN), (is, VBZ),...","[[(a, DT), (crime, NN), (scene, NN)], [[('is',...",SOV


In [34]:
like_sov_text = like_sov_df['Far_Chunks']

In [36]:
for item in like_sov_text[:10]:
    print(item)
    print()
    print()

[قلعه كاپريكورن NP] [مثل PP] [يک هتل NP] [5 ستاره NP] [است VP]


[هي آدم حقه NP] [باز، PP] [فكر مي NP] [كني VP] [كجا ADVP] [داري PP] [ميري؟ NP]


[اون زن NP] [مثل PP] [يک فرشته NP] [الهيه VP] که [به PP] [اين دنيا مهربوني ميبخشه NP]


[For sunlight is like gold NP] [براي PP] [نور NP] [خورشيد VP] که [به PP] [رنگ طلاست NP]


[صحنه NP] [جرم NP] [رازهاي VP] [زيادي ADVP] [توش داره VP]


[قدرت NP] [مثل PP] [دانه NP] [هاي VP] [شنه NP]


[من NP] [ميدونم كه زندگي NP] [بدون PP] [پشتيبان NP] [چه جون NP] [كندني VP] ميشه


[اين NP] [مثل PP] [سرپرست هست، NP] [کسي VP] که [سخت ADVP] [کار مي NP] [کنه VP] تا [شمشير NP] [درست ADJP] [کنه VP] .


[تمرینات بانوی پزشک یانگوم NP] [در PP] [پزشکی NP] [مانند PP] [کار یک مادر NP] [است VP]


[داگو NP] [مثل PP] [پدر من NP] [میمونه VP] [زود ADJP] [برمیگردم VP]




#### Looking at the SVO Texts Above:

In [37]:
like_svo_text = like_svo_df['Far_Chunks']

In [38]:
for item in like_svo_text[:10]:
    print(item)
    print()
    print()

[اين ويزيتها NP] [شبيه VP] [اغاز يه مسابقه تلوزيونيه NP]


[اين NP] [آهنگه VP] [مثله خوره ميمونه NP]


[اين NP] [هست VP] [هنر کفش NP]


[کريسمس NP] [توي VP] [کورس من NP] [بين VP] [مامان NP] و [بابا NP] [خلاصه ميشه NP]


[عشقمون NP] [شبيه VP] [يک گل رز قرمزه NP]


[زندگي اين پايين NP] [شبيه VP] [زندگي تو NP] [فضاست VP] [نبايد VP] [زياد اشتباه NP] [کني VP]


[رفتار NP] [هاي VP] [شرم آور VP] [درست شبيه اينه كه خون شهدا NP] [را POSTP] [پايمال كنيم NP]


[مردن NP] [بعنوان شهيد شبيه VP] [تزريق خون NP] [در PP] [شَريانه جامعه NP] [ست VP]


[شوخي ميکني اينجور NP] [چيزا براي VP] [من NP] [مثل PP] [آب ميمونن NP]


[تمرين كردن NP] [براي VP] [من NP] [مثل PP] [يک مذهب ميمونه NP] [خالي نميبندم VP]




#### Looking at some of the "Other" Data:

In [41]:
other_df = pd.read_pickle('other_only_df.pkl')

In [42]:
len(other_df)

535371

In [43]:
other_df.head()

,ID,Eng,Far,Eng_Tok,Far_Tok,Eng_Len,Far_Len,Eng_Types,Far_Types,Far_POS,Far_Chunks,Eng_POS,Eng_Chunks,Word_Order
0,1,raspy breathing,صداي خر خر,"[raspy, breathing]","[صداي, خر, خر]",2,3,"{breathing, raspy}","{صداي, خر}","[(صداي, NUM), (خر, Ne), (خر, N)]",[صداي خر خر NP],"[(raspy, NN), (breathing, NN)]","[[(raspy, NN), (breathing, NN)]]",Other
1,2,dad,پدر,[dad],[پدر],1,1,{dad},{پدر},"[(پدر, N)]",[پدر NP],"[(dad, NN)]","[[(dad, NN)]]",Other
2,3,maybe its the wind,شايد صداي باد باشه,"[maybe, its, the, wind]","[شايد, صداي, باد, باشه]",4,4,"{its, wind, maybe, the}","{شايد, باد, صداي, باشه}","[(شايد, Ne), (صداي, AJ), (باد, V), (باشه, V)]",[شايد صداي NP] [باد VP] [باشه VP],"[(maybe, RB), (its, PRP$), (the, DT), (wind, NN)]","[(maybe, RB), (its, PRP$), [(the, DT), (wind, ...",Other
3,4,no,نه,[no],[نه],1,1,{no},{نه},"[(نه, ADV)]",نه,"[(no, DT)]","[[(no, DT)]]",Other
5,6,you have a week evans then well burn the house,اوانز تو فقط يک هفته وقت داري وگرنه خونتو خواه...,"[you, have, a, week, evans, then, well, burn, ...","[اوانز, تو, فقط, يک, هفته, وقت, داري, وگرنه, خ...",10,11,"{well, the, house, burn, evans, a, then, have,...","{اوانز, داري, خواهيم, سوزوند, خونتو, وقت, يک, ...","[(اوانز, Ne), (تو, PRO), (فقط, ADV), (يک, Ne),...",[اوانز تو NP] [فقط يک هفته وقت ADVP] [داري VP]...,"[(you, PRP), (have, VBP), (a, DT), (week, NN),...","[(you, PRP), [[('have', 'VBP')], [('a', 'DT'),...",Other


In [44]:
other_sample = other_df['Far_Chunks']

In [47]:
for item in other_sample.sample(10):
    print(item)
    print()

[تا PP] [وقتي كه اشراف جلسه تشكيل بدن NP]

. [ممنونم VP] [چارلي NP]

[از PP] [وقتى NP] که [من NP] [به PP] [اون خونه NP] [رفتم VP] [من NP] [هم ADVP] [يک همچين چيزايى NP] [را POSTP] [دارم VP] [تجربه ميکنم NP]

[اونها NP] [نسبت به PP] [اون پسر خشمگين شدن NP]

[حالا ADVP] [ما NP] [تخم هارو داغ و خوب ميکنيم اينجوري سفت و محکم ميشن NP]

[پس ADVP] [تو NP] [چند ماهه وضع خوبي NP] [نداشتي VP] [تو NP] [هر كاري كه NP] [بقيه PP] [ميكنن انجام ميدي NP]

و [من NP] [اون‌و NP] [اين شکلي VP] مي‌بينم

[به PP] [چه علت NP] [گذاشتيد VP] [بوول بياد VP] [بين شما NP] ، [توليور NP] ، [هان آروم NP] [باش VP] ، [اون اومد VP] [پيش من NP]

[پس ADVP] [یانگوم NP] و [مینگ جو NP] [از PP] [آنجا NP] [همدیگر NP] [را POSTP] [می شناسند؟ VP]

[غمزده عشق NP]

